In [1]:
'''
    @Author: King
    @Date: 2019.05.16
    @Purpose: Spark入门：键值对RDD(Python版)
    @Introduction:  spark 实现键值对RDD(Python版)
    @Datasets: 
    @Link : 
    @Reference : http://dblab.xmu.edu.cn/blog/1706-2/
'''

'\n    @Author: King\n    @Date: 2019.05.16\n    @Purpose: Spark入门：键值对RDD(Python版)\n    @Introduction:  spark 实现键值对RDD(Python版)\n    @Datasets: \n    @Link : \n    @Reference : http://dblab.xmu.edu.cn/blog/1706-2/\n'

## Spark入门：键值对RDD(Python版)

![作者](../img/bigdata-roadmap.jpg)
【版权声明】博客内容由厦门大学数据库实验室拥有版权，未经允许，请勿转载！

虽然RDD中可以包含任何类型的对象，但是“键值对”是一种比较常见的RDD元素类型，分组和聚合操作中经常会用到。

Spark操作中经常会用到“键值对RDD”（Pair RDD），用于完成聚合计算。普通RDD里面存储的数据类型是Int、String等，而“键值对RDD”里面存储的数据类型是“键值对”。

## 准备工作
1. 启动 pyspark

In [1]:
from pyspark import SparkContext
sc = SparkContext( 'local', 'test')

### 键值对RDD的创建
#### 第一种创建方式：从文件中加载
我们可以采用多种方式创建键值对RDD，其中一种主要方式是使用map()函数来实现，如下：

In [20]:
lines = sc.textFile("word.txt")
pairRDD = lines.flatMap(lambda line : line.split()).map(lambda word : (word,1))
pairRDD.foreach(print)
pairRDD.first()
print (pairRDD.collect())

[('面', 1), ('第', 1), ('一', 1), ('行', 1), ('首', 1), ('先', 1), ('从', 1), ('外', 1), ('部', 1), ('文', 1), ('件', 1), ('d', 1), ('a', 1), ('t', 1), ('a', 1), ('.', 1), ('t', 1), ('x', 1), ('t', 1), ('中', 1), ('构', 1), ('建', 1), ('得', 1), ('到', 1), ('一', 1), ('个', 1), ('R', 1), ('D', 1), ('D', 1), ('，', 1), ('名', 1), ('称', 1), ('为', 1), ('l', 1), ('i', 1), ('n', 1), ('e', 1), ('s', 1)]


#### 第二种创建方式：通过并行集合（列表）创建RDD

In [32]:
list = ["Hadoop","Spark","Hive","Spark"]
rdd = sc.parallelize(list)
airRDD = rdd.map(lambda word : (word,1))
print (airRDD.collect())

[('Hadoop', 1), ('Spark', 1), ('Hive', 1), ('Spark', 1)]


## 常用的键值对转换操作

常用的键值对转换操作包括reduceByKey()、groupByKey()、sortByKey()、join()、cogroup()等，下面我们通过实例来介绍。

### 1.reduceByKey(func)

reduceByKey(func)的功能是，使用func函数合并具有相同键的值。比如，reduceByKey((a,b) => a+b)，有四个键值对(“spark”,1)、(“spark”,2)、(“hadoop”,3)和(“hadoop”,5)，对具有相同key的键值对进行合并后的结果就是：(“spark”,3)、(“hadoop”,8)。可以看出，(a,b) => a+b这个Lamda表达式中，a和b都是指value，比如，对于两个具有相同key的键值对(“spark”,1)、(“spark”,2)，a就是1，b就是2。

我们对上面第二种方式创建得到的pairRDD进行reduceByKey()操作，代码如下：

In [33]:
airRDD = airRDD.reduceByKey(lambda a,b : a+b)
print (airRDD.collect())

[('Hadoop', 1), ('Spark', 2), ('Hive', 1)]


### 2.groupByKey()

groupByKey()的功能是，对具有相同键的值进行分组。比如，对四个键值对(“spark”,1)、(“spark”,2)、(“hadoop”,3)和(“hadoop”,5)，采用groupByKey()后得到的结果是：(“spark”,(1,2))和(“hadoop”,(3,5))。
我们对上面第二种方式创建得到的pairRDD进行groupByKey()操作，代码如下：

In [34]:
groupRDD = pairRDD.groupByKey()
print (groupRDD.collect())

[('面', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AADD8>), ('第', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AA160>), ('一', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AAB70>), ('行', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AA978>), ('首', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AAF28>), ('先', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AABE0>), ('从', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AAE10>), ('外', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AACC0>), ('部', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AABA8>), ('文', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AAE48>), ('件', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AA860>), ('d', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AA240>), ('a', <pyspark.resultiterable.ResultIterable object at 0x0000023D7F9AAC88>)

### 3.keys()

keys()只会把键值对RDD中的key返回形成一个新的RDD。比如，对四个键值对(“spark”,1)、(“spark”,2)、(“hadoop”,3)和(“hadoop”,5)构成的RDD，采用keys()后得到的结果是一个RDD[Int]，内容是{“spark”,”spark”,”hadoop”,”hadoop”}。

我们对上面第二种方式创建得到的pairRDD进行keys操作，代码如下：

In [35]:
keyRDD = pairRDD.keys()
print (keyRDD.collect())

['面', '第', '一', '行', '首', '先', '从', '外', '部', '文', '件', 'd', 'a', 't', 'a', '.', 't', 'x', 't', '中', '构', '建', '得', '到', '一', '个', 'R', 'D', 'D', '，', '名', '称', '为', 'l', 'i', 'n', 'e', 's']


### 4.values()

values()只会把键值对RDD中的value返回形成一个新的RDD。比如，对四个键值对(“spark”,1)、(“spark”,2)、(“hadoop”,3)和(“hadoop”,5)构成的RDD，采用values()后得到的结果是一个RDD[Int]，内容是{1,2,3,5}。

我们对上面第二种方式创建得到的pairRDD进行values()操作，代码如下：

In [38]:
valuesRDD = pairRDD.values()
print (valuesRDD.collect())

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### 5.sortByKey()

sortByKey()的功能是返回一个根据键排序的RDD。

我们对上面第二种方式创建得到的pairRDD进行keys操作，代码如下：

In [39]:
sortByKeyRDD = pairRDD.sortByKey()
print (sortByKeyRDD.collect())

[('.', 1), ('D', 1), ('D', 1), ('R', 1), ('a', 1), ('a', 1), ('d', 1), ('e', 1), ('i', 1), ('l', 1), ('n', 1), ('s', 1), ('t', 1), ('t', 1), ('t', 1), ('x', 1), ('一', 1), ('一', 1), ('个', 1), ('中', 1), ('为', 1), ('从', 1), ('件', 1), ('先', 1), ('到', 1), ('名', 1), ('外', 1), ('建', 1), ('得', 1), ('文', 1), ('构', 1), ('称', 1), ('第', 1), ('行', 1), ('部', 1), ('面', 1), ('首', 1), ('，', 1)]


### 6.mapValues(func)

我们经常会遇到一种情形，我们只想对键值对RDD的value部分进行处理，而不是同时对key和value进行处理。对于这种情形，Spark提供了mapValues(func)，它的功能是，对键值对RDD中的每个value都应用一个函数，但是，key不会发生变化。比如，对四个键值对(“spark”,1)、(“spark”,2)、(“hadoop”,3)和(“hadoop”,5)构成的pairRDD，如果执行pairRDD.mapValues(lambda x : x+1)，就会得到一个新的键值对RDD，它包含下面四个键值对(“spark”,2)、(“spark”,3)、(“hadoop”,4)和(“hadoop”,6)。

我们对上面第二种方式创建得到的pairRDD进行keys操作，代码如下：

In [40]:
mapValuesRDD = pairRDD.mapValues(lambda x : x+1)
print (mapValuesRDD.collect())

[('面', 2), ('第', 2), ('一', 2), ('行', 2), ('首', 2), ('先', 2), ('从', 2), ('外', 2), ('部', 2), ('文', 2), ('件', 2), ('d', 2), ('a', 2), ('t', 2), ('a', 2), ('.', 2), ('t', 2), ('x', 2), ('t', 2), ('中', 2), ('构', 2), ('建', 2), ('得', 2), ('到', 2), ('一', 2), ('个', 2), ('R', 2), ('D', 2), ('D', 2), ('，', 2), ('名', 2), ('称', 2), ('为', 2), ('l', 2), ('i', 2), ('n', 2), ('e', 2), ('s', 2)]


### 7.join

join(连接)操作是键值对常用的操作。“连接”(join)这个概念来自于关系数据库领域，因此，join的类型也和关系数据库中的join一样，包括内连接(join)、左外连接(leftOuterJoin)、右外连接(rightOuterJoin)等。最常用的情形是内连接，所以，join就表示内连接。

对于内连接，对于给定的两个输入数据集(K,V1)和(K,V2)，只有在两个数据集中都存在的key才会被输出，最终得到一个(K,(V1,V2))类型的数据集。

比如，pairRDD1是一个键值对集合{(“spark”,1)、(“spark”,2)、(“hadoop”,3)和(“hadoop”,5)}，pairRDD2是一个键值对集合{(“spark”,”fast”)}，那么，pairRDD1.join(pairRDD2)的结果就是一个新的RDD，这个新的RDD是键值对集合{(“spark”,1,”fast”),(“spark”,2,”fast”)}。对于这个实例，我们下面在pyspark中运行一下：

In [41]:
pairRDD1 = sc.parallelize([('spark',1),('spark',2),('hadoop',3),('hadoop',5)])
pairRDD2 = sc.parallelize([('spark','fast')])
pairRDD3 = pairRDD1.join(pairRDD2)
print (pairRDD3.collect())

[('spark', (1, 'fast')), ('spark', (2, 'fast'))]


### 一个综合实例

题目：给定一组键值对(“spark”,2),(“hadoop”,6),(“hadoop”,4),(“spark”,6)，键值对的key表示图书名称，value表示某天图书销量，请计算每个键对应的平均值，也就是计算每种图书的每天平均销量。

很显然，对于上面的题目，结果是很显然的，(“spark”,4),(“hadoop”,5)。

下面，我们在pyspark中演示代码执行过程：

In [43]:
rdd = sc.parallelize([("spark",2),("hadoop",6),("hadoop",4),("spark",6)])
rdd.mapValues(lambda x : (x,1)).reduceByKey(lambda x,y : (x[0]+y[0],x[1] + y[1])).mapValues(lambda x : (x[0] / x[1])).collect()

[('spark', 4.0), ('hadoop', 5.0)]

要注意，上面语句中，mapValues(lambda x : (x,1))中出现了变量x，reduceByKey(lambda x,y : (x[0]+y[0],x[1]+ y[1]))中也出现了变量x，mapValues(lambda x : (x[0] / x[1]))也出现了变量x。但是，必须要清楚，这三个地方出现的x，虽然都具有相同的变量名称x，但是，彼此之间没有任何关系，它们都处在不同的变量作用域内。如果你觉得这样会误导自己，造成理解上的掌握，实际上，你可以把三个出现x的地方分别替换成x1、x2、x3也是可以的，但是，很显然没有必要这么做。

上面是完整的语句和执行过程，可能不太好理解，下面我们进行逐条语句分解给大家介绍。每条语句执行后返回的屏幕信息，可以帮助大家更好理解语句的执行效果，比如生成了什么类型的RDD。

（1）首先构建一个数组，数组里面包含了四个键值对，然后，调用parallelize()方法生成RDD，从执行结果反馈信息，可以看出，rdd类型是RDD[(String, Int)]。

In [44]:
rdd = sc.parallelize([("spark",2),("hadoop",6),("hadoop",4),("spark",6)])

（2）针对构建得到的rdd，我们调用mapValues()函数，把rdd中的每个每个键值对(key,value)的value部分进行修改，把value转换成键值对(value,1)，其中，数值1表示这个key在rdd中出现了1次，为什么要记录出现次数呢？因为，我们最终要计算每个key对应的平均值，所以，必须记住这个key出现了几次，最后用value的总和除以key的出现次数，就是这个key对应的平均值。比如，键值对(“spark”,2)经过mapValues()函数处理后，就变成了(“spark”,(2,1))，其中，数值1表示“spark”这个键的1次出现。下面就是rdd.mapValues()操作在spark-shell中的执行演示：
```
scala> rdd.mapValues(x => (x,1)).collect()
res23: Array[(String, (Int, Int))] = Array((spark,(2,1)), (hadoop,(6,1)), (hadoop,(4,1)), (spark,(6,1)))
```

上面语句中，collect()是一个行动操作，功能是以数组的形式返回数据集中的所有元素，当我们要实时查看一个RDD中的元素内容时，就可以调用collect()函数。

（3）然后，再对上一步得到的RDD调用reduceByKey()函数，在spark-shell中演示如下：

In [45]:
rdd.mapValues(lambda x : (x,1)).reduceByKey(lambda x,y : (x[0]+y[0],x[1] + y[1])).collect()

[('spark', (8, 2)), ('hadoop', (10, 2))]

这里，必须要十分准确地理解reduceByKey()函数的功能。可以参考上面我们对该函数的介绍，reduceByKey(func)的功能是使用func函数合并具有相同键的值。这里的func函数就是Lamda表达式 x,y : (x[0]+y[0],x[1] + y[1])，这个表达式中，x和y都是value，而且是具有相同key的两个键值对所对应的value，比如，在这个例子中， (“hadoop”,(6,1))和(“hadoop”,(4,1))这两个键值对具有相同的key，所以，对于函数中的输入参数(x,y)而言，x就是(6,1)，序列从0开始计算，x[0]表示这个键值对中的第1个元素6，x[1]表示这个键值对中的第二个元素1，y就是(4,1)，y[0]表示这个键值对中的第1个元素4，y[1]表示这个键值对中的第二个元素1，所以，函数体(x[0]+y[0],x[1] + y[2])，相当于生成一个新的键值对(key,value)，其中，key是x[0]+y[0]，也就是6+4=10，value是x[1] + y[1]，也就是1+1=2，因此，函数体(x[0]+y[0],x[1] + y[1])执行后得到的value是(10,2)，但是，要注意，这个(10,2)是reduceByKey()函数执行后，”hadoop”这个key对应的value，也就是，实际上reduceByKey()函数执行后，会生成一个键值对(“hadoop”,(10,2))，其中，10表示hadoop书籍的总销量，2表示两天。同理，reduceByKey()函数执行后会生成另外一个键值对(“spark”,(8,2))。

(4)最后，就可以求出最终结果。我们可以对上面得到的两个键值对(“hadoop”,(10,2))和(“spark”,(8,2))所构成的RDD执行mapValues()操作，得到每种书的每天平均销量。当第一个键值对(“hadoop”,(10,2))输入给mapValues(x => (x[0] / x[1]))操作时，key是”hadoop”，保持不变，value是(10,2)，会被赋值给Lamda表达式x => (x[0] / x[1]中的x，因此，x的值就是(10,2)，x[0]就是10，表示hadoop书总销量是10，x[1]就是2，表示2天，因此，hadoop书籍的每天平均销量就是x[0] / x[1]，也就是5。mapValues()输出的一个键值对就是(“hadoop”,5)。同理，当把(“spark”,(8,2))输入给mapValues()时，会计算得到另外一个键值对(“spark”,4)。在pyspark中演示如下：

In [46]:
rdd.mapValues(lambda x: (x,1)).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).mapValues(lambda x: (x[0]/x[1])).collect()

[('spark', 4.0), ('hadoop', 5.0)]